Depoly counter contract

In [7]:
from web3 import Web3
from eth_account import Account
import json
import os
from dotenv import load_dotenv
load_dotenv()

# Connect to Hyperliquid testnet
w3 = Web3(Web3.HTTPProvider('https://api.hyperliquid-testnet.xyz/evm'))

# Your private key (keep this secret and safe!)
private_key = os.getenv('DEPLOYER_PRIVATE_KEY')
account = Account.from_key(private_key)

# Load compiled contract data
with open('./bin/Counter.abi', 'r') as f:
    abi = json.loads(f.read())
    
with open('./bin/Counter.bin', 'r') as f:
    bytecode = '0x' + f.read()

# Create contract instance
Counter = w3.eth.contract(abi=abi, bytecode=bytecode)

# Build transaction
transaction = Counter.constructor().build_transaction({
    'from': account.address,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 1000000,
    'gasPrice': w3.eth.gas_price
})

# Sign transaction
signed = w3.eth.account.sign_transaction(transaction, private_key)

# Send transaction
tx_hash = w3.eth.send_raw_transaction(signed.raw_transaction)  # Changed from rawTransaction to raw_transaction

# Wait for transaction receipt
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Contract deployed at: {tx_receipt.contractAddress}")

Contract deployed at: 0xfD09670395Da97077dC885D0897fc3046C4ac8D1


Test the deployed contract

In [9]:
# Create contract instance at deployed address
contract_address = '0xfD09670395Da97077dC885D0897fc3046C4ac8D1'  # The address from your deployment
counter = w3.eth.contract(address=contract_address, abi=abi)

# Read initial count (call function)
initial_count = counter.functions.getCount().call()  # Changed from get() to getCount()
print(f"Initial count: {initial_count}")

# Increment count (send transaction)
increment_txn = counter.functions.increment().build_transaction({
    'from': account.address,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 100000,
    'gasPrice': w3.eth.gas_price
})

# Sign and send transaction
signed_txn = w3.eth.account.sign_transaction(increment_txn, private_key)
tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Wait for transaction to be mined
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Increment transaction successful! Hash: {tx_hash.hex()}")

# Read new count
new_count = counter.functions.getCount().call()  # Changed from get() to getCount()
print(f"New count: {new_count}")

Initial count: 0
Increment transaction successful! Hash: 66743e9d14fece37816782eb280c7317f47e8674812cb15322ed4dcef449d890
New count: 1
